_Warning to the Jupyter reader: Gist/Github drop some formula lines and ignore MathJax backslash newlines; Latex doesn't render Greek characters in source blocks; and Notebook doesn't support \mathbb. The only export format that remotely works is HTML. What a clusterfuck._

In [1]:
# %matplotlib widget for interactive mode, or
# %matplotlib inline for print mode
%matplotlib widget
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

ASPECT = 1.4
mpl.rc('figure',
       figsize=(8, 8/ASPECT),  # inches
       dpi=100)

Introduction
===

This problem has been discussed on
[Computer Science Stack Exchange](https://cs.stackexchange.com/questions/127815). To recap:

There are up to 25 nodes, each representing a
[Terraria](https://en.wikipedia.org/wiki/Terraria) NPC's home location. These locations can be optimized based on a collection of cost factors. All factors are cost multipliers (higher is worse). Distances are discrete integers. None of the factors are continuous, and none are differentiable in space since they are all piece-wise. As a result, something like Gradient Descent is not possible without continuous approximation.

The 2D coordinates of each node are discrete and - for these purposes - unbounded. Since there are 25 nodes, there are 50 integer variables (xy for each node) to optimize. The hope is that even though there are no bounds, there will be enough sub-1.0 factors to have the optimization converge rather than force the nodes to fly apart.

In most modes of this optimization problem there are also between 2-3 biome regions extending out infinitely from the origin separated by straight lines. Each biome exactly encompasses one or more Cartesian quadrants. Each imposes a fixed cost or benefit to each node, but in the ideal case this cost does not change the "farther into the biome" a node gets. If I get the base optimization working well enough for a given biome configuration, I might expand this to selection of a biome configuration, for which there are currently 34 possibilities.

Cost Analysis
===

Simple Pair Costs
---

In the ideal (discrete) case, simple cost factors such as NPC pair relationships mentioned in the
[wiki](https://terraria.gamepedia.com/NPCs#Happiness):

> - For each Loved NPC within 25 tiles: 90%
> - For each Liked NPC within 25 tiles: 95%
> - For each Disliked NPC within 25 tiles: 105%
> - For each Hated NPC within 25 tiles: 110%

can be represented by a modified Heaviside function for each other node, e.g.:

$$
\mu = 1 + (1.05 - 1) H(25 - d)
$$

In [2]:
d = np.linspace(0, 40, 200)
μ = 1 + (1.05 - 1)*np.heaviside(25 - d, 0.5)
fig = plt.figure(1)
plt.plot(d, μ)
fig.axes[0].set_axisbelow(True)
plt.title('"Disliked" cost, ideal Heaviside')
plt.xlabel('Distance between these two nodes')
plt.ylabel('Cost factor')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Sigmoid Approximation
---

However, this cost function is inconvenient, because it prevents any kind of continuous optimizer from working. It is non-differentiable and so standard techniques like gradient descent will not work. Instead, the suggested approach is to make a continuous sigmoid approximation. Increasing values of $\alpha$ will improve the approximation:

$$
\mu = 1 + \lim_{\alpha \to \infty} 
\frac {1.05 - 1}
{1 + e^{\alpha (d - 25)}}
$$

In [3]:
α = np.array([0.2, 0.3, 0.5, 1, 4], ndmin=2)
d = np.array(np.linspace(0, 40, 500), ndmin=2).T
μ = 1 + (1.05 - 1)/(1 + np.exp(α*(d - 25)))
fig = plt.figure(2)
plt.plot(d, μ)
fig.axes[0].set_axisbelow(True)
plt.title('"Disliked" cost, approximate sigmoid')
plt.xlabel('Distance between these two nodes')
plt.ylabel('Cost factor')
plt.legend(α.flatten(), title='α')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

With four example NPCs that are liked, disliked, loved and hated (respectively), this sigmoid approximation applies in two dimensions:

In [4]:
α = 2
Y = 60

other_nodes = np.array((
    (-25, 20),
    ( 15, 11),
    (  0, -4),
    ( 21,-30),
))
node_rels = np.array((0.95, 1.05, 0.90, 1.10))
x, y = np.meshgrid(
    np.linspace(-Y*ASPECT, Y*ASPECT, 200),
    np.linspace(-Y, Y, 200),
)
grid = np.stack((x, y), axis=2)[:, :, np.newaxis, :]
node_dists = np.linalg.norm(
    grid - other_nodes, axis=3
)
μ = (
    1 + (node_rels - 1)/(
        1 + np.exp(α*(node_dists - 25))
    )
).prod(axis=2)

fig = plt.figure(3)
plot = plt.contourf(x, y, μ, levels=20)
fig.axes[0].set_axisbelow(True)
plt.title(f'Pair cost, approximate sigmoid, α={α}')
plt.colorbar()
plt.scatter(other_nodes[:, 0], other_nodes[:, 1], c='k')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Biome costs
---

A node incurs a cost or benefit to being in some biomes:

> - Located in a Loved biome: 90%
> - Located in a Liked biome: 95%
> - Located in a Disliked biome: 105%
> - Located in a Hated biome: 110%

The cost function depends on the calculated quadrant, since the optimisation will assume that the town is centered:

- in one biome;
- on the edge of two biomes beside or on top of one another; or
- at the corner of three biomes, where each biome exactly covers at least one Cartesian quadrant.

$$
\theta \in \text{I}, 0 \le \theta < 4
$$

$$
\theta=\mod\left(
   \left\lfloor \frac {2 \text{atan2}(y,x)} \pi \right\rfloor,
   4
\right)
$$

or as a piece-wise function,

$$
\theta = \begin{cases}
  0  & x \ge 0, y \ge 0 \\
  1  & x <   0, y \ge 0 \\
  2  & x <   0, y <   0 \\
  3  & x \ge 0, y <   0 \\
\end{cases}
$$

Biome costs for each quadrant are then

$$b_{\theta} \in \text{R}, b_{\theta} > 0$$

For the ideal case,

$$\mu = b_{\theta}$$

For the approximated case, the cost factor must rely on all four quadrants. First we need to calculate or hard-code the quadrant signs:

$$
s_x(\theta) = \sqrt{2} \cos \frac {\pi(\theta+0.5)} 2 
= (+1, -1, -1, +1)
$$

$$
s_y(\theta) = \sqrt{2} \sin \frac {\pi(\theta+0.5)} 2 
= (+1, +1, -1, -1)
$$

The approximated cost is then

$$
\mu(x, y) = \prod_{\theta=0}^3
\left(1 + 
    \lim_{\alpha \to \infty}
    \frac {b_\theta - 1}
    {
        \left(1 + e^{-\alpha x s_x(\theta)} \right)
        \left(1 + e^{-\alpha y s_y(\theta)} \right)
    }
\right)
$$

By example, for _liked_ , _hated_ , _neutral_ and _neutral_ biomes in quadrants I through IV respectively:

$$b = (0.95, 1.10, 1.00, 1.00)$$

In [5]:
α = 2
Y = 5
x, y = np.meshgrid(
    np.linspace(-Y*ASPECT, Y*ASPECT, 200),
    np.linspace(-Y, Y, 200),
)
b  = np.array((0.95,  1.10,  1.00,  1.00), ndmin=3)
sx = np.array((1, -1, -1,  1), ndmin=3)
sy = np.array((1,  1, -1, -1), ndmin=3)

μ = (
    1 + (b - 1)
    / (1 + np.exp(-α*sx * x[:, :, np.newaxis]))
    / (1 + np.exp(-α*sy * y[:, :, np.newaxis]))
).prod(axis=2)

fig = plt.figure(4)
plot = plt.contour(x, y, μ, levels=15)
fig.axes[0].set_axisbelow(True)
plt.title(f'Biome cost, approximate sigmoid, α={α}')
plt.clabel(plot)
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Crowding Cost
---

Multi-node interactions will be complex:

> Two or more other NPCs within 25 tiles (for each additional NPC): 104%

For a given node being optimized, for each potential pair of coordinates being considered, relative to each other node, calculate the Frobenius norm. Subtract a radius of 25 from the norm and use that for the sigmoid exponent. Get the product $u$ of all such sigmoids for the current node. This will not yet take into account the _two or more_ clause.

$$
u(x, y) =
\lim_{\alpha_1 \to \infty}
\prod_i \left(
    1 + 
    \frac {1.04 - 1}
    {
        1 + e^{
            \alpha_1( ||m_i - n||_F - 25 )
        }
    }
\right)
$$

Divide by 1.04 to discount the effect of only one proximate node. Apply an outer parametric
[softplus](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#Softplus)
rectification to make the minimum of this cost component 1.0.

$$
\mu(x, y) = 1 + 
\lim_{\alpha_2 \to \infty}
\frac 1 {\alpha_2}
\ln \left(
    1 + \exp \left(
        \alpha_2 \left(
            \frac {u(x, y)} {1.04} - 1
        \right)
    \right)
\right)
$$

$\alpha_2$ must be on the order of 100 since its coefficient (multiples of 0.04) is so low.

In [6]:
α1 = 2
α2 = 100
C = 1.04
Y = 35

other_nodes = np.array((
    (-25, 20),
    ( 15, 11),
    (  0, -4),
    ( 21,-30),
))
x, y = np.meshgrid(
    np.linspace(-Y*ASPECT, Y*ASPECT, 200),
    np.linspace(-Y, Y, 200),
)
grid = np.stack((x, y), axis=2)[:, :, np.newaxis, :]
node_dists = np.linalg.norm(
    grid - other_nodes, axis=3
)
u = (
    1 + (C - 1)/(
        1 + np.exp(α1*(node_dists - 25))
    )
).prod(axis=2)
μ = 1 + np.log(1 + np.exp(α2*(u/C - 1)))/α2

fig = plt.figure(5)
plot = plt.contourf(x, y, μ, levels=20)
fig.axes[0].set_axisbelow(True)
plt.title(f'Crowding cost, approximate sigmoid, α₁={α1}, α₂={α2}')
plt.colorbar()
plt.scatter(other_nodes[:, 0], other_nodes[:, 1], c='k')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In Figure 4, the current node is not shown since its position is varying for optimisation. Close to the outer two nodes, the cost is nearly 1.0, and there are visible steps to 1.04 and 1.04² where the radii of two and three nodes overlap, respectively.

Complex Crowding Cost
---

> No more than one other NPC within 25 tiles and no more than three other NPCs (not counting the NPCs within 25 tiles) within 120 tiles: 90%

This implies:

- Outside of the area near NPCs, the cost will be 0.9
- If there is one other NPC within 25 tiles, and three other NPCs between 25 and 120 tiles, the cost is still 0.9
- If any of the above figures are exceeded, the cost increases to 1.0

In exact terms, this cost could be expressed based on $a$, the number of NPCs within 25 tiles, and $b$, the number of NPCS between 25 and 120 tiles away:

$$
\mu = 0.9 + (1-0.9)
\min\left(
    1, 
    \max(0,a-1) + \max(0,b-3)
\right)
$$

In approximate terms,

$$
a(x,y) = -1 +
\lim_{\alpha_1 \to \infty}
\sum_i
\frac 1 {
    1 + e^{\alpha_1 (
        ||m_i - n||_F - 25
    )}
}
$$

$$
b(x,y) = -3 +
\lim_{\alpha_1 \to \infty}
\sum_i
\frac 1 {
    \left(
        1 + e^{\alpha_1 (
            25 - ||m_i - n||_F
        )}
    \right)
    \left(
        1 + e^{\alpha_1 (
            ||m_i - n||_F - 120
        )}
    \right)
}
$$

$a$ and $b$ range from -1 and -3 respectively, upward in steps of 1; and need to receive respective minima of 0, be summed, and then receive a collective maximum of 1, for a total of three softplus applications:

$$
\text{smin}(x) =
\lim_{\alpha_2 \to \infty}
\frac 1 {\alpha_2}
\ln \left(
    1 + e^{\alpha_2 x}
\right)
$$

$$
\text{smax}(x) = 1 -
\lim_{\alpha_3 \to \infty}
\frac 1 {\alpha_3}
\ln \left(
    1 + e^{\alpha_3 (1-x)}
\right)
$$
        
$$
\mu(x,y) = 0.9 + 
(1 - 0.9)
\text{smax} \left(
    \text{smin}(a(x,y)) + \text{smin}(b(x,y))
\right)
$$

In [7]:
α1 = 0.5
α2 = 75
α3 = 75
C = 0.9
Y = 120

other_nodes = np.array((
    (-20, 30),
    ( 20, 21),
    (  5,  6),
    ( 26,-20),
))
x, y = np.meshgrid(
    np.linspace(-Y*ASPECT, Y*ASPECT, 200),
    np.linspace(-Y, Y, 200),
)
grid = np.stack((x, y), axis=2)[:, :, np.newaxis, :]
node_dists = np.linalg.norm(
    grid - other_nodes, axis=3
)

a = (
    1
    /(1 + np.exp(α1*(node_dists - 25)))
).sum(axis=2) - 1
b = (
    1
    /(1 + np.exp(-α1*(node_dists - 25)))
    /(1 + np.exp(α1*(node_dists - 120)))
).sum(axis=2) - 3

with_min = (
    np.log(
        1 + np.exp(
            α2 * np.stack((a, b))
        )
    ) / α2
).sum(axis=0)
with_max = 1 - np.log(
    1 + np.exp(
        α3*(1 - with_min)
    )
) / α3
μ = C + (1 - C)*with_max

fig = plt.figure(6)
plot = plt.contourf(x, y, μ, levels=20)
plt.colorbar()
fig.axes[0].set_axisbelow(True)
plt.title(f'Complex crowding cost, approximate sigmoid, α₁={α1}, α₂={α2}, α₃={α3}')
plt.scatter(other_nodes[:, 0], other_nodes[:, 1], c='k')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Due to the strange logic associated with this cost, an equally strange pattern is produced with problematic local minima. It is expensive to be within 120 tiles of four NPCs; but as soon as you approach to the point where one of those NPCs is within 25 tiles, the cost temporarily decreases.

Putting it all together
---

Based on these:

> - The below price modifiers are multiplicative
> - Factors that make an NPC happy will lower its prices for goods purchased from it, down to a minimum of 75%
> - Factors that make an NPC unhappy will raise its prices purchased from it, up to a maximum of 150%
> - Price modifiers are rounded to the nearest 5% increment

The cost components need to be multiplied, receive a double softplus, and then receive a "soft-staircase".

The softplus limits are possible via

$$
0.75 + \frac 1 \alpha \left[
  \ln \left(
    1 + e^{\alpha(x - 0.75)}
  \right)
  - \ln \left(
    1 + e^{\alpha(x - 1.5)}
  \right)
\right]
$$

The staircase as suggested by 
[this post](https://math.stackexchange.com/a/2049337/54983)
would be

$$
0.05 \left[
  \frac {
    \tanh \left(
      \alpha \left(
        \frac x {0.05}
        - \left\lfloor \frac x {0.05} \right\rfloor
        - \frac 1 2
      \right)
    \right)
  }
  {
    2 \tanh ( \alpha / 2 )
  }
  + \frac 1 2
  + \left\lfloor
    \frac x {0.05}
  \right\rfloor
\right]
$$

with $\alpha > 5$ for increasing sharpness.